In [1]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

from itertools import permutations
from IPython.display import display, HTML

import numpy as np
import pandas as pd
from copy import deepcopy
from Merge import merge
from copy import deepcopy
from Logic import logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from Battery import Battery
from DPModel import DPModel, DP, DP_stochastic

from P2P_Dynamics import EnergyMarket

In [2]:
Start = '2022-06-19 00:00:00'
End = '2022-06-19 23:00:00'
N = len(pd.date_range(Start, End,freq='H'))

dfA = merge('h16')
dfB = merge('h28')
dfC = merge('k28')

In [3]:
battery = Battery(max_capacity=13, max_charge=7)
series_DP_A = DP(Start, End, dfA, battery, byday=True, ints=True, degrade=False, verbose=True)
series_DP_B = DP(Start, End, dfB, Battery(max_capacity=13, max_charge=7,current_capacity=10), byday=True, ints=True, degrade=False, verbose=True)
series_DP_C = DP(Start, End, dfC, battery, byday=True, ints=True, degrade=False, verbose=True)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00


In [4]:
print_price_summary(series_DP_A, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  150.0  DKK
Total emissions for period:  13.0  kg

Number of kwh purchased in the period: 72.3
Number of kwh sold in the period: 6.5


In [5]:
print_price_summary(series_DP_B, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  70.0  DKK
Total emissions for period:  8.0  kg

Number of kwh purchased in the period: 38.7
Number of kwh sold in the period: 5.999999999999999


In [6]:
print_price_summary(series_DP_C, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  173.0  DKK
Total emissions for period:  14.0  kg

Number of kwh purchased in the period: 80.10000000000001
Number of kwh sold in the period: 0.0


In [7]:
class DP_central(DPModel):
    def __init__(self, Start, End, merges, houses, battery,degrade=False,ints=False,acts=None,acts_range=None,
                 furthers=None,traj=None, traj_range=None,max_number_states=200): 
        super().__init__(Start, End, merges[0], battery,degrade,ints,acts,acts_range)
        self.merges = merges
        self.houses = houses
        self.furthers = furthers
        self.traj = traj
        self.traj_range = traj_range
        self.max_number_states = max_number_states
        
        temp = pd.DataFrame(columns=houses)
        for i in range(len(houses)):
            temp[houses[i]] = merges[i].loc[Start:End]["yield"]

        self.yields = temp
    
        #Compute state space once
        if ints:
            states = np.arange(0, battery.max_capacity+1, 1)
        else:
            states = np.round(np.arange(0.0, battery.max_capacity+0.01, 0.1),2)
        
        self.states = [states for _ in range(len(houses))]
        
        if self.ints and self.furthers is not None:
            for i in range(len(houses)):
                temp = self.states[i]
                self.states[i] = temp[temp%self.furthers[i][0]==self.furthers[i][1]]

        self.s = np.array(np.meshgrid(*self.states)).T.reshape(-1,3)
        self.s = self.s.tolist()
        self.s = [tuple(x) for x in self.s]
    
    def f(self, x, u, w, k):
        ogbat = self.battery
        self.battery = deepcopy(ogbat)
        
        res = []
        for i in range(len(self.houses)):
            self.battery.current_capacity = x[i]
            self.battery.charge(u[i], degrade=self.degrade)

            if self.ints:
                self.battery.current_capacity = int(self.battery.current_capacity)
            
            res.append(self.battery.get_current_capacity())
            
        self.battery = ogbat
        
        return tuple(res)
    
    def g(self, x, u, w, k):
        yields = self.get_yield(k)
        surpluses = [yields[i]-u[i] for i in range(len(u))]
        
        fee = 1 #transmission fee
        
        participants = {self.houses[i]: surpluses[i] for i in range(len(u))}
    
        em = EnergyMarket(participants, self.sp[k], self.sp[k]+fee)
        
        dic = em.get_total_costs()
        
        return sum([dic[house] for house in self.houses])
    
    def S(self, k):
        if (self.traj is not None) and (self.traj_range is not None):
            for i in range(len(self.houses)):
                if self.traj_range[i]==0.0:
                    self.states[i] = self.traj[k][i]
                
            temp = np.array(np.meshgrid(*self.states)).T.reshape(-1,3)
            temp = temp.tolist()
            temp = [tuple(x) for x in temp]
            
            return temp
        
        return self.s
    
    def A(self, x, k):
        states=[]
        acts = self.acts
        self.acts=None
        for i in range(len(x)):
            temp = super().A(x[i], k)
            states.append(temp)
            
        self.acts=acts
        
        if self.ints:
            states = [states[i][np.array(states[i],dtype=int)==states[i]] for i in range(len(x))]
            
        if self.acts is not None and self.acts_range is not None:
            for i in range(len(x)):
                if self.acts_range[i]!=0.0:
                    below =states[i][states[i]<=self.acts[k][i]][-int((self.acts_range[i]+0.1)*10):]
                    above =states[i][states[i]>self.acts[k][i]][:int(self.acts_range[i]*10)]

                    states[i] = np.append(below,above)
                else:
                    states[i] = self.acts[k][i]
        
        if self.ints and self.furthers is not None:
            states = [states[i][(states[i]+x[i])%self.furthers[i][0]==self.furthers[i][1]] for i in range(len(x))]
            
        actions = np.array(np.meshgrid(*states)).T.reshape(-1,3)
        if len(actions)>self.max_number_states:
            idx = np.round(np.linspace(0, len(actions) - 1, self.max_number_states)).astype(int)
            actions = actions[idx]

        actions = list(map(tuple,actions))
        return actions
    
    def get_yield(self,k):
        return self.yields.iloc[k].to_numpy()

In [8]:
class DP_P2P:
    def __init__(self, start_time, end_time, merges, houses, battery):
        if len(houses)<=1:
            raise Exception("P2P requires more than one house!")
        if False in [house in ["k28", "h16", "h22", "h28", "h32"] for house in houses]:
            raise Exception('All houses should be either "k28", "h16", "h22", "h28", or "h32"')
        if type(battery) is not Battery:
            raise Exception("battery must be a Battery class instance!")
            
        self.start_time = start_time
        self.end_time = end_time
        self.merges = merges
        self.houses = houses
        
        self.battery = battery
        self.N = len(pd.date_range(start=start_time,end=end_time,freq="h"))
        
        #For P2P_sol
        self.results = None
        self.results_ord = None
        
        #For cost_matrix
        self.nf = None
        
        #For all_sol
        self.all_results = None
        self.all_results_ord = None
        self.all_nf = None
        self.all_costs = None
        self.best_idx = None
        
        self.sp = merges[0].loc[Start:End]["SpotPriceDKK"]/1000
        self.ep = merges[0].loc[Start:End]["CO2Emission"]/1000
        
        all_houses = ["h16", "h22", "h28", "h32", "k28"] 
        self.all_houses = all_houses
        
        houses_ord = []
        merges_ord = []
        i=0
        for house in self.all_houses:
            if house in self.houses:
                houses_ord.append(house)
                merges_ord.append(merges[i])
                i+=1
                
        self.houses_ord = houses_ord
        self.merges_ord = merges_ord

        
        #Permutation information
        perms = list(permutations([i+1 for i in range(len(self.houses))]))
        
        house_rewrites = [[self.houses_ord[i-1] for i in perms[j]] for j in range(len(perms))]
        merges_rewrites = [[self.merges_ord[i-1] for i in perms[j]] for j in range(len(perms))]
        
        
        perms_names = [''.join(str(i) for i in perms[j]) for j in range(len(perms))]
        
        self.perms = perms
        self.house_rewrites = house_rewrites
        self.merges_rewrites = merges_rewrites
        self.perms_names = perms_names
        
        #Find this permutation
        self.this_idx, self.this_perm, self.this_perm_name = self.find_this_perm()
        
        
        self.constraints = None

    def find_constraints(self, x0):
        constraints = []
        for i in range(len(self.houses)):
            bat_copy = deepcopy(battery)
            bat_copy.current_capacity = x0[i]
            series = DP(self.start_time, self.end_time, self.merges[i], bat_copy, 
                        byday=True, ints=True, degrade=False, verbose=False)
            constraints.append(series["cost"].sum())
            
        self.constraints = constraints
        
    def find_this_perm(self):
        for i, houses in enumerate(self.house_rewrites):
            if tuple(houses)==tuple(self.houses):
                this_idx = i
                this_perm = self.perms[this_idx]
                this_perm_name = self.perms_names[this_idx]   
                return this_idx, this_perm, this_perm_name
        
    def tup_add(self,tup1,tup2):
        res = [int((tup1[i]+tup2[i])*10)/10 for i in range(len(tup1))] 
        return tuple(res)

    def correct_traj_acts(self,x0,traj,acts):
        traj[0]=x0
        for i in range(len(acts)):
            traj[i+1]=self.tup_add(traj[i],acts[i])

        return traj
    
    def p2p_rollout(self,model, pi, x0):
        cost = 0
        surpluses = []
        J, x, trajectory, actions = 0, x0, [x0], []
        for k in range(model.N):
            u = pi(x, k)
            price = model.g(x, u , True, k)
            surpluses.append([model.get_yield(k)[i]-u[i] for i in range(len(u))])
            J+=price

            x = model.f(x, u, True, k)
            trajectory.append(x) # update the list of the trajectory
            actions.append(u) # update the list of the actions

        J += model.gN(x)
        return J, trajectory, actions, np.array(surpluses)
    
    def run(self, j, Start, End, x0, x0_int, max_number_states, trajectory=None, actions=None, ints=False):
        if ints:    
            furthers=[[1,0]]
            for t in range(len(self.houses)-1):
                furthers.append([self.battery.max_capacity+1,x0_int[t+1]])

            DPP2P= DP_central(Start, End, self.merges, self.houses, deepcopy(self.battery),
                              degrade=False,ints=True,acts=None,acts_range=None,
                              furthers=furthers,traj=None,traj_range=None, max_number_states=max_number_states)
            _, pi = DP_stochastic(DPP2P)
            
            J, trajectory, actions, surpluses = self.p2p_rollout(model=DPP2P, pi=lambda x, k: pi[k][x], x0=x0_int)

            if x0 != x0_int:
                trajectory = self.correct_traj_acts(x0,trajectory,actions)

        elif (trajectory is not None) and (actions is not None):
            traj_range = [0 if t!=j+1 else -1 for t in range(len(self.houses))]  
            acts_range = [0 if t!=j+1 else self.battery.max_charge for t in range(len(self.houses))]

            DPP2P= DP_central(Start, End, self.merges, self.houses, deepcopy(self.battery),
                              degrade=False,ints=False,acts=actions,acts_range=acts_range,
                              furthers=None,traj=trajectory,traj_range=traj_range, max_number_states=max_number_states)
            _, pi = DP_stochastic(DPP2P)

            J, trajectory, actions, surpluses = self.p2p_rollout(model=DPP2P, pi=lambda x, k: pi[k][x], x0=x0)

        else:
            raise Exception("Must have either furthers or trajectory and actions as input!")

        return J, trajectory, actions, surpluses
        
    def P2P_sol(self, x0, max_number_states=20, brute_push = False, byday=True, verbose=True):
        N=self.N

        all_actions = []
        all_surpluses = np.ones((0,len(self.houses)))

        Start_i = self.start_time
        x0_i = x0
        x0_int_i = tuple([int(x0[i]) for i in range(len(x0))])

        num_loops = int(np.ceil(N/24)) if byday else 1
        remainder = N%24
        length = 24 if byday else N
        for i in range(num_loops):
            if byday and i == num_loops-1:
                length = length if remainder == 0 else remainder

            End_i = pd.date_range(start=Start_i,periods=length,freq="h")[-1]

            if verbose:
                print(f"Period from {Start_i} to {End_i}")
            
            furthers=[[1,0]]
            for t in range(len(self.houses)-1):
                furthers.append([self.battery.max_capacity+1,x0_int_i[t+1]])
            
            for j in range(len(self.houses)):
                if j==0:
                    J, trajectory, actions, surpluses = self.run(j, Start_i, End_i, x0_i, x0_int_i, max_number_states, 
                                                                 trajectory=None, actions=None, ints=True)
                        
                J, trajectory, actions, surpluses = self.run(j, Start_i, End_i, x0_i, x0_int_i, max_number_states, 
                                                             trajectory=trajectory, actions=actions, ints=False)
       
            trajectory_og, actions_og, surpluses_og = deepcopy(trajectory), deepcopy(actions), np.copy(surpluses)
           
            if brute_push:
                while True:
                    if verbose:
                        print("Brute forcing another round")
                    for j in range(len(self.houses)):       
                        temp, trajectory, actions, surpluses = self.run(j, Start_i, End_i, x0_i, x0_int_i, max_number_states, 
                                                                        trajectory=trajectory, actions=actions, ints=False)

                    J_prev=J
                    J = temp
                    if verbose:
                        print(f"J_prev = {J_prev}, J = {J}")
                        print()
                    if J>=J_prev:
                        trajectory, actions, surpluses = trajectory_og, actions_og, surpluses_og
                        break

                    trajectory_og, actions_og, surpluses_og = deepcopy(trajectory), deepcopy(actions), np.copy(surpluses)
                
            
            all_actions = all_actions + actions
            all_surpluses = np.append(all_surpluses,surpluses,axis=0)

            Start_i= pd.date_range(start=End_i,periods=2,freq="h")[-1]
            
            x0_i = trajectory[-1]
            x0_int_i = tuple([int(x0_i[t]) for t in range(len(x0_i))])

        self.results = (all_actions,all_surpluses)
        
        
        all_surpluses_ord = np.copy(all_surpluses)
        all_actions_ord = np.array(all_actions)
        
        _,this_perm,_ = self.find_this_perm()
        
        for j, i in enumerate(this_perm):
            all_surpluses_ord[:,i-1] = all_surpluses[:,j]
            all_actions_ord[:,i-1] = np.array(all_actions)[:,j]
        

        all_actions_ord = list(map(tuple,all_actions_ord))
        
        self.results_ord =(all_actions_ord,all_surpluses_ord)
        
        return self.results_ord
    
    def cost_matrix(self):
        if self.results is None:
            return None
        
        _,surpluses_ord = self.results_ord
        
        nf = pd.DataFrame()
        for i,house in enumerate(self.houses_ord):
            nf[house] = surpluses_ord[:,i]
            
        costs = pd.DataFrame()
        emissions = pd.DataFrame()
        for i in range(len(nf)):
            temp = pd.DataFrame(EnergyMarket(nf.iloc[i].to_dict(),self.sp[i],self.sp[i]+1).get_total_costs(), index=[0])
            costs = pd.concat([costs,temp],ignore_index=True)
            
            temp = pd.DataFrame(EnergyMarket(nf.iloc[i].to_dict(),-1,self.ep[i],True).get_total_costs(), index=[0])
            emissions = pd.concat([emissions,temp],ignore_index=True)
            
        for house in self.houses_ord:
            nf['cost_'+house] = costs[house].to_list()
        
        for house in self.houses_ord:
            nf['cumm_cost_'+house] = nf['cost_'+house].cumsum()
                
        for house in self.houses_ord:
            nf['emis_'+house] = emissions[house].to_list()
            
        for house in self.houses_ord:
            nf['cumm_emis_'+house] = nf['emis_'+house].cumsum()
         
        self.nf = nf
        
        return nf
    
    def total_cost(self):
        if self.nf is None:
            return None
        
        return sum([self.nf['cumm_cost_'+house][len(self.nf)-1] for house in self.houses])
    
    def all_sol(self, x0, max_number_states=20, brute_push = False, byday=True, verbose=True):
        x0_ord = list(self.this_perm)
        for j, i in enumerate(self.this_perm):
            x0_ord[i-1] = x0[j]
            
        x0_ord = tuple(x0_ord)
        
        all_results = []
        all_results_ord = []
        all_nf = []
        all_costs = []
            
        if verbose:
            print(f"This permutation is called '{self.this_perm_name}'. Starting with permutation '{self.perms_names[0]}'")
            print()
            
        for i in range(len(self.house_rewrites)):
            if verbose:
                print(f"Permutation {self.perms_names[i]}, ({i+1}/{len(self.perms_names)})")
            
            x0 = tuple([x0_ord[j-1] for j in self.perms[i]])
            self.houses = self.house_rewrites[i]
            self.merges = self.merges_rewrites[i]
            
            self.P2P_sol(x0, max_number_states=max_number_states, brute_push = brute_push, byday=byday, verbose=verbose)
            self.cost_matrix()
            
            all_results.append(self.results)
            all_results_ord.append(self.results_ord)
            all_nf.append(self.nf)
            all_costs.append(self.total_cost())

            if verbose:
                print()
        
        #reset
        self.houses = self.house_rewrites[self.this_idx]
        self.merges = self.merges_rewrites[self.this_idx]
        
        self.all_results = all_results
        self.all_results_ord = all_results_ord
        self.all_nf = all_nf
        self.all_costs = all_costs
        
        self.best_idx = np.argmin(all_costs)
        
        if verbose:
            print("Done!")
            print()
            print(f"Best found permutation was {self.perms_names[self.best_idx]} with cost {self.all_costs[self.best_idx]}")
           
    def sol_print(self):
        if self.nf is None:
            return
        
        display(HTML(self.nf._repr_html_()))
        
    def all_sol_print(self):
        if self.all_nf is None:
            return
        
        ognf = self.nf
        for i in range(len(stuff.all_nf)):
            self.nf = self.all_nf[i]
            print(f"This is the cost matrix for sol with permutation {self.perms_names[i]}")
            self.sol_print()
            print(f"Total cost this permutation = {self.all_costs[i]}")
            print()
            print()
            print()
            
        #reset
        self.nf = ognf
            
    def all_sol_actions(self):
        if self.all_results_ord is None:
            return
        df = pd.DataFrame()
        for i in range(len(self.all_results_ord)):
            df["actions"+self.perms_names[i]] = self.all_results_ord[i][0]
        return df
    
    def equivalent(self, model, traj, acts, pi, seen_surpluses, result):
        if result is not None:
            return result, seen_surpluses
        start = len(traj)-1
        x = traj[-1]
        for k in range(start,len(acts),1):
            u = acts[k]
            price = model.g(x, u , True, k)

            perms = set(permutations(u))
            perms.remove(u)
            if len(perms)>0:
                A = model.A(traj[k-1],k-1)
                for i, a in enumerate(perms):
                    if a in A:
                        if model.g(x, a, True, k)-price<=0.001: 
                            pi[k][x] = a
                            traj_copy = deepcopy(traj)
                            x = model.f(x, a, True, k)
                            traj_copy.append(x)
                            acts_copy = deepcopy(acts)
                            acts_copy[k] = a


                            result, seen_surpluses = self.equivalent(model, traj_copy, acts_copy, pi, seen_surpluses, result)
                            if result is not None:
                                return result, seen_surpluses

            pi[k][x] = u
            x = model.f(x, u, True, k)
            traj.append(x) # update the list of the trajectory

        _, _, actions, surpluses = self.p2p_rollout(model, lambda x,k: pi[k][x], traj[0])

        if surpluses.tolist() in seen_surpluses:
            return None, seen_surpluses
        
        seen_surpluses.append(surpluses.tolist())
        
        surpluses_ord = np.copy(surpluses)
        actions_ord = np.array(actions)
        
        results_ord_og = self.results_ord
        nf_og = self.nf
        
        for j, i in enumerate(self.this_perm):
            surpluses_ord[:,i-1] = surpluses[:,j]
            actions_ord[:,i-1] = np.array(actions)[:,j]


        actions_ord = list(map(tuple,actions_ord))

        self.results_ord = (actions_ord,surpluses_ord)
        nf = self.cost_matrix()
        
        checks = [nf['cumm_cost_'+house][len(nf)-1]<=self.constraints[i] for i,house in enumerate(self.houses)]
        
        print(checks)
        print()
        
        if not (False in checks):
            #print("if u didn't see a checks idk wtf is going on tbh")
            #print(checks)
            #print(not (False in checks))
            self.result = (actions, surpluses)
            return self.result, seen_surpluses
        
        self.sol_print()
        print()
        print()
        
        #reset
        self.results_ord = results_ord_og
        self.nf = nf_og
        
        return None, seen_surpluses
    
    def find_good_result(self, x0):
        if self.results is None:
            return None
        
        actions, _ = self.results
        
        traj = [x0]
        
        DPP2P = DP_central(Start, End, merges, houses, battery,degrade=False,ints=False,acts=None,acts_range=None,
                 furthers=None,traj=None, traj_range=None,max_number_states=float('inf')) 
        
        pi =  [{} for _ in range(len(actions))]
        return self.equivalent(DPP2P, traj, actions, pi, [], None)

In [9]:
bat = Battery(max_capacity=13,max_charge=7)
max_number_states=20
houses = ["h16","h28","k28"]
merges = [merge(house) for house in houses]
x0 = (0,10,0)

In [10]:
stuff = DP_P2P(Start,End,merges,houses,bat)
stuff.find_constraints(x0)

In [11]:
stuff.all_sol(x0, max_number_states=20, brute_push = False, byday=True, verbose=True)

This permutation is called '123'. Starting with permutation '123'

Permutation 123, (1/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Permutation 132, (2/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Permutation 213, (3/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Permutation 231, (4/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Permutation 312, (5/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Permutation 321, (6/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Done!

Best found permutation was 213 with cost 382.3877183001099


In [12]:
stuff.all_sol_print()

This is the cost matrix for sol with permutation 123


,h16,h28,k28,cost_h16,cost_h28,cost_k28,cumm_cost_h16,cumm_cost_h28,cumm_cost_k28,emis_h16,emis_h28,emis_k28,cumm_emis_h16,cumm_emis_h28,cumm_emis_k28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847,0.7535,-0.3014,0.6987,0.7535,-0.3014,0.6987
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330,0.5406,0.0306,0.6426,1.2941,-0.2708,1.3413
2,-4.2,-3.3,-5.6,9.371460,7.363290,12.495280,36.134678,4.698455,44.044611,0.6132,0.4818,0.8176,1.9073,0.2110,2.1589
3,-3.8,-2.8,-4.1,7.751886,5.711916,8.363877,43.886564,10.410371,52.408487,0.5890,0.4340,0.6355,2.4963,0.6450,2.7944
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,51.095696,16.101791,60.186762,0.5890,0.4650,0.6355,3.0853,1.1100,3.4299
5,-10.4,-4.9,-3.5,18.984472,8.944607,6.389005,70.080168,25.046398,66.575767,1.9240,0.9065,0.6475,5.0093,2.0165,4.0774
6,3.5,-0.4,-3.2,-2.930410,0.434904,2.679232,67.149758,25.481302,69.254999,-0.8435,0.0241,0.0000,4.1658,2.0406,4.0774
7,-3.1,-2.4,-4.3,5.373106,4.159824,7.453018,72.522864,29.641126,76.708017,0.8184,0.6336,1.1352,4.9842,2.6742,5.2126
8,-10.3,-9.6,-9.8,17.236535,16.065120,16.399810,89.759399,45.706246,93.107827,2.8531,2.6592,2.7146,7.8373,5.3334,7.9272
9,1.8,-0.5,-1.9,-1.262430,0.850675,1.432565,88.496970,46.556921,94.540392,-0.4716,0.1310,0.0262,7.3657,5.4644,7.9534


Total cost this permutation = 382.38771830011



This is the cost matrix for sol with permutation 132


,h16,h28,k28,cost_h16,cost_h28,cost_k28,cumm_cost_h16,cumm_cost_h28,cumm_cost_k28,emis_h16,emis_h28,emis_k28,cumm_emis_h16,cumm_emis_h28,cumm_emis_k28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847,0.7535,-0.3014,0.6987,0.7535,-0.3014,0.6987
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330,0.5406,0.0306,0.6426,1.2941,-0.2708,1.3413
2,-4.2,-3.3,-5.6,9.371460,7.363290,12.495280,36.134678,4.698455,44.044611,0.6132,0.4818,0.8176,1.9073,0.2110,2.1589
3,-3.8,-2.8,-4.1,7.751886,5.711916,8.363877,43.886564,10.410371,52.408487,0.5890,0.4340,0.6355,2.4963,0.6450,2.7944
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,51.095696,16.101791,60.186762,0.5890,0.4650,0.6355,3.0853,1.1100,3.4299
5,-10.4,-2.7,-5.7,18.984472,4.928661,10.404951,70.080168,21.030452,70.591713,1.9240,0.4995,1.0545,5.0093,1.6095,4.4844
6,3.5,-2.6,-1.0,-2.930410,2.176876,0.937260,67.149758,23.207328,71.528973,-0.8435,0.0000,0.0241,4.1658,1.6095,4.5085
7,-3.1,-2.4,-4.3,5.373106,4.159824,7.453018,72.522864,27.367152,78.981991,0.8184,0.6336,1.1352,4.9842,2.2431,5.6437
8,-10.3,-9.6,-9.8,17.236535,16.065120,16.399810,89.759399,43.432272,95.381801,2.8531,2.6592,2.7146,7.8373,4.9023,8.3583
9,1.8,-1.2,-1.2,-1.262430,0.841620,1.441620,88.496970,44.273892,96.823421,-0.4716,0.0000,0.1572,7.3657,4.9023,8.5155


Total cost this permutation = 382.38771830011007



This is the cost matrix for sol with permutation 213


...

Total cost this permutation = 382.38771830011



This is the cost matrix for sol with permutation 312


,h16,h28,k28,cost_h16,cost_h28,cost_k28,cumm_cost_h16,cumm_cost_h28,cumm_cost_k28,emis_h16,emis_h28,emis_k28,cumm_emis_h16,cumm_emis_h28,cumm_emis_k28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847,0.7535,-0.3014,0.6987,0.7535,-0.3014,0.6987
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330,0.5406,0.0306,0.6426,1.2941,-0.2708,1.3413
2,-4.2,-3.3,-5.6,9.371460,7.363290,12.495280,36.134678,4.698455,44.044611,0.6132,0.4818,0.8176,1.9073,0.2110,2.1589
3,-3.8,-2.8,-4.1,7.751886,5.711916,8.363877,43.886564,10.410371,52.408487,0.5890,0.4340,0.6355,2.4963,0.6450,2.7944
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,51.095696,16.101791,60.186762,0.5890,0.4650,0.6355,3.0853,1.1100,3.4299
5,-5.6,-2.7,-10.5,10.222408,4.928661,19.167015,61.318104,21.030452,79.353776,1.0360,0.4995,1.9425,4.1213,1.6095,5.3724
6,-1.3,-2.6,3.8,1.188438,2.176876,-3.181588,62.506542,23.207328,76.172188,0.0241,0.0000,-0.9158,4.1454,1.6095,4.4566
7,-3.1,-2.4,-4.3,5.373106,4.159824,7.453018,67.879648,27.367152,83.625206,0.8184,0.6336,1.1352,4.9638,2.2431,5.5918
8,-10.3,-9.6,-9.8,17.236535,16.065120,16.399810,85.116184,43.432272,100.025017,2.8531,2.6592,2.7146,7.8169,4.9023,8.3064
9,-3.0,-1.2,3.6,2.104050,1.441620,-2.524860,87.220233,44.873892,97.500157,0.0000,0.1572,-0.9432,7.8169,5.0595,7.3632


Total cost this permutation = 382.38771830010995



This is the cost matrix for sol with permutation 321


,h16,h28,k28,cost_h16,cost_h28,cost_k28,cumm_cost_h16,cumm_cost_h28,cumm_cost_k28,emis_h16,emis_h28,emis_k28,cumm_emis_h16,cumm_emis_h28,cumm_emis_k28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847,0.7535,-0.3014,0.6987,0.7535,-0.3014,0.6987
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330,0.5406,0.0306,0.6426,1.2941,-0.2708,1.3413
2,-4.2,-3.3,-5.6,9.371460,7.363290,12.495280,36.134678,4.698455,44.044611,0.6132,0.4818,0.8176,1.9073,0.2110,2.1589
3,-3.8,-2.8,-4.1,7.751886,5.711916,8.363877,43.886564,10.410371,52.408487,0.5890,0.4340,0.6355,2.4963,0.6450,2.7944
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,51.095696,16.101791,60.186762,0.5890,0.4650,0.6355,3.0853,1.1100,3.4299
5,-3.4,-4.9,-10.5,6.206462,8.944607,19.167015,57.302158,25.046398,79.353776,0.6290,0.9065,1.9425,3.7143,2.0165,5.3724
6,-3.5,-0.4,3.8,2.930410,0.434904,-3.181588,60.232568,25.481302,76.172188,0.0000,0.0241,-0.9158,3.7143,2.0406,4.4566
7,-3.1,-2.4,-4.3,5.373106,4.159824,7.453018,65.605674,29.641126,83.625206,0.8184,0.6336,1.1352,4.5327,2.6742,5.5918
8,-10.3,-9.6,-9.8,17.236535,16.065120,16.399810,82.842210,45.706246,100.025017,2.8531,2.6592,2.7146,7.3858,5.3334,8.3064
9,-3.7,-0.5,3.6,2.694995,0.850675,-2.524860,85.537204,46.556921,97.500157,0.0262,0.1310,-0.9432,7.4120,5.4644,7.3632


Total cost this permutation = 382.38771830010995





In [13]:
df = stuff.all_sol_actions()
df

,actions123,actions132,actions213,actions231,actions312,actions321
0,"(0.0, -7.0, -0.0)","(0.0, -7.0, -0.0)","(-0.0, -7.0, -0.0)","(-0.0, -7.0, -0.0)","(-0.0, -7.0, 0.0)","(-0.0, -7.0, 0.0)"
1,"(0.0, -3.0, -0.0)","(0.0, -3.0, -0.0)","(-0.0, -3.0, -0.0)","(-0.0, -3.0, -0.0)","(-0.0, -3.0, 0.0)","(-0.0, -3.0, 0.0)"
2,"(0.0, -0.0, -0.0)","(0.0, -0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, -0.0, 0.0)","(-0.0, -0.0, 0.0)"
3,"(0.0, -0.0, -0.0)","(0.0, -0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, -0.0, 0.0)","(-0.0, -0.0, 0.0)"
4,"(0.0, -0.0, -0.0)","(0.0, -0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, -0.0, 0.0)","(-0.0, -0.0, 0.0)"
5,"(7.0, 2.2, -0.0)","(7.0, -0.0, 2.2)","(2.2, 7.0, -0.0)","(-0.0, 7.0, 2.2)","(2.2, -0.0, 7.0)","(-0.0, 2.2, 7.0)"
6,"(-7.0, -2.2, -0.0)","(-7.0, -0.0, -2.2)","(-2.2, -7.0, -0.0)","(-0.0, -7.0, -2.2)","(-2.2, -0.0, -7.0)","(-0.0, -2.2, -7.0)"
7,"(0.0, -0.0, -0.0)","(0.0, -0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, 0.0, -0.0)","(-0.0, -0.0, 0.0)","(-0.0, -0.0, 0.0)"
8,"(7.0, 7.0, 7.0)","(7.0, 7.0, 7.0)","(7.0, 7.0, 7.0)","(7.0, 7.0, 7.0)","(7.0, 7.0, 7.0)","(7.0, 7.0, 7.0)"
9,"(-5.0, -0.2, 0.5)","(-5.0, 0.5, -0.2)","(-0.2, -5.0, 0.5)","(0.5, -5.0, -0.2)","(-0.2, 0.5, -5.0)","(0.5, -0.2, -5.0)"
